In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import  mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
from dnn_model import DNN_Model

In [2]:
# Load validation data

validation_data_path = 'data/right_breast_validation_top_20_features_data.csv'
validation_data = pd.read_csv(validation_data_path)

# Split data into predictors and target
X_val = validation_data.iloc[:, :-1].values
y_true = validation_data.iloc[:, -1].values

# Load training data for scaler
training_data_path = 'data/train_top_20_features_data.csv'
training_data = pd.read_csv(training_data_path)

# Split data into predictors (X) and target (y)
X_train = training_data.iloc[:, :-1].values

# Scale features 
scaler = MinMaxScaler()  
X_train = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [3]:
#Loading model
model_path = 'models/dose_pred_dnn/breast_dose_pred_model.pth'
model = DNN_Model(input_size=X_val.shape[1], hidden_sizes=[32, 16, 8], output_size=1, num_layers=4, dropout=0.1)
model.load_state_dict(torch.load(model_path))
model.eval() 

# Convert validation data to torch Tensor
X_validation_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    y_pred = model(X_validation_tensor).numpy().flatten()

In [5]:
# Calculating evaluation metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred) *100

# Calculating percentage differences
percentage_diff = np.abs((y_true - y_pred) / y_true) * 100
max_percentage_diff = np.max(percentage_diff)
min_percentage_diff = np.min(percentage_diff)

# Creating a pandas dataframe to display the results
metrics = {
    'Metric': ['Mean Absolute Error', 'Mean Squared Error', 'Mean Absolute Percentage Error', 'Max Percentage Error', 'Min Percentage Error'],
    'Value': [mae, mse, mape, max_percentage_diff, min_percentage_diff]
}

df_metrics = pd.DataFrame(metrics)
df_metrics['Value'] = df_metrics['Value'].round(2)

# Displaying the dataframe
title = 'Results - Right Breast'
print(title)
df_metrics


Results - Right Breast


,Metric,Value
0,Mean Absolute Error,0.20
1,Mean Squared Error,0.06
2,Mean Absolute Percentage Error,4.02
3,Max Percentage Error,18.70
4,Min Percentage Error,0.04
